<a href="https://colab.research.google.com/github/N-nolwenn/SAM/blob/main/Copie_de_TP1_2023_index_fichier_ETU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SAM: TP1 Accès aux données avec index 

Sujet pour étudiants

date de modification : 26/01/2023 16h

NOM: BOUCHOUCHI et PIGEON

Prénom: Nour et Nolwenn

Objectifs:
Savoir organiser des données en pages pour permettre de modifier un tuple en ne modifiant qu'une seule page.

Comprendre les méthodes d'accès suivantes :

*   Lecture séquentielle d'un fichier : "table access full"
*   Lecture d'un tuple dont on connait le rowid : "table access by index rowid"
*   Opération de sélection par lecture séquentielle et filtrage 

Comprendre les méthodes d'indexation :

*   Créer un index
*   Opération de Sélection par index et lecture par rowid

Mise à jour de données
*   Sélectionner un tuple et modifier un de ses attributs
*   Modifier l'index en conséquence lorsque l'attibut modifié est indexé

Persistence
*   Stocker un index (dans plusieurs pages) pour le reconstruire plus rapidement
*   Adapter en conséquence les opérations de modification de l'index


In [ ]:
import os
import shutil as sh
import numpy as np
import random
from random import choice
from string import ascii_lowercase
import time

DATA = "data.csv"

# Générer un fichier

Création du fichier

In [ ]:
# dure environ 40s pour 5M lignes

nb_lines = 5 * 1000 * 1000
# nb_lines = 100
nb_attributes = 7

longueur_attribut = 100
# string_val = "".join(choice(ascii_lowercase) for i in range(longueur_attribut))
long_string = ''.join('-' for i in range(longueur_attribut))

# a=[np.random.randint(0, int(nb_lines/(10**i)), nb_lines) for i in range(nb_attributes)]
nb_valeurs_distinctes = nb_lines

# le premier attribut est unique
a = [random.sample(range(nb_valeurs_distinctes), nb_lines)]

# les attributs suivants ont des domaines plus petits
for i in range(1, nb_attributes):
  nb_valeurs_distinctes = max(2, int(nb_valeurs_distinctes / 2))
  a.append(np.random.randint(0, nb_valeurs_distinctes, nb_lines))

b = [ ','.join(map(lambda x: str(x), e)) + f",{long_string}\n" for e in zip(*a)]

with open(DATA, "w") as f:
  f.write(''.join(b))

In [ ]:
%%bash
echo "head : "
head -n 2 data.csv
echo "tail : "
tail -n 2 data.csv
echo "size (lines) :"
wc -l data.csv

head : 
2177963,1784542,139688,422469,132238,2709,12294,----------------------------------------------------------------------------------------------------
854935,1643638,988548,320588,19415,111698,71256,----------------------------------------------------------------------------------------------------
tail : 
1192258,1792173,620908,85620,213834,99783,62734,----------------------------------------------------------------------------------------------------
3610775,2152521,1149698,289963,212576,49904,77794,----------------------------------------------------------------------------------------------------
size (lines) :
5000000 data.csv


# Lecture séquentielle

In [ ]:
def filtrer_fichier(fichier, valeur_recherchee):
  with open(fichier, "r") as f:
    for i, line in enumerate(f):
      a = int(line.split(',')[0]) #cle unique
      if a == s :
        print(f"ligne {i} :", line.strip())


s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_fichier(DATA, s)
print("done in", time.time() - t1, "s")

valeur recherchée : 50084
ligne 4604232 : 50084,2392862,705280,499270,125832,98193,26855,----------------------------------------------------------------------------------------------------
done in 4.8410539627075195 s


# Découper le fichier en pages

In [ ]:
def page_dir_name(fichier):
  return fichier.split('.')[0] + "_pages"

def decoupe_fichier_en_pages(fichier, nb_tuple_par_page):
  page_dir = page_dir_name(fichier)
  print("pages dans :", page_dir)
  if(os.path.exists(page_dir)):
    sh.rmtree(page_dir)
  os.makedirs(page_dir, exist_ok=True)

  with open(fichier, "r") as f:
    p=0
    lines = []
    for i, line in enumerate(f):
      lines.append(line)
      if (i+1) % nb_tuple_par_page == 0:
        p += 1
        with open(page_dir + f"/page{p}", "w") as fp:
          fp.write(''.join(lines))
        lines = []
    if len(lines) > 0:
      p +=1
      with open(page_dir + f"/page{p}", "w") as fp:
          fp.write(''.join(lines))
    
    print("nb pages créées :", p)

decoupe_fichier_en_pages(DATA, nb_tuple_par_page=1000)

pages dans : data_pages
nb pages créées : 5000


Afficher le nombre de tuples dans une page (pour quelques pages)

In [ ]:
%%bash
wc -l data_pages/* | head -n 3

     1000 data_pages/page1
     1000 data_pages/page10
     1000 data_pages/page100


# Lecture séquentielle du fichier découpé en pages

In [ ]:
def lecture_sequentielle_par_page(fichier):
   page_dir = page_dir_name(fichier)
   nb_pages = len(os.listdir(page_dir))
   
   # a faire : pour chaque page, lire ses lignes
   # une ligne devient un tuple
   # retourner un itérateur contenant le numéro de page, la position dans la page et le tuple
   for p in range(nb_pages) :
     with open(page_dir + f"/page{(p+1)}", "r") as fp:
       for i, line in enumerate(fp):
         t = line.split(',')
         yield p+1,i+1,t
   


def filtrer_fichier_par_pages(fichier, valeur_recherchee):
  # à faire pour chaque (numéro de page, position dans la page, tuple) obtenu en invoquant la méthode ci-dessus
  # convertir le 1er attribut en un nombre l'afficher si il est egal à la valeur recherchee  
  y = lecture_sequentielle_par_page(fichier)
  for line in y:
    a = int(line[2][0])
    if a == valeur_recherchee : 
      print(line)



s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
filtrer_fichier_par_pages("data.txt", s)
print("done in", round(time.time() - t1, 2), "s")

valeur recherchée : 40112
(882, 165, ['40112', '710238', '167279', '246916', '4401', '109424', '58494', '----------------------------------------------------------------------------------------------------\n'])
done in 5.51 s


# Lecture d'un tuple dans une page

In [ ]:
def lecture_tuple(fichier, num_page, position):
  page_dir = page_dir_name(fichier)
  with open(page_dir + f"/page{num_page}", "r") as fp:
    for i, line in enumerate(fp):
      if(i==position-1):
        #print(line)  
        return line


t1 = time.time()
lecture_tuple("data.txt", 188,6)
print("done in", round(time.time() - t1, 2), "s")

done in 0.0 s


# Créer un index

In [ ]:
def creation_index_unique(fichier):
  index = {}

  # la clé est la valeur du 1er attribut
  # la valeur est un rowid composé de (page, position)
  y = lecture_sequentielle_par_page(fichier)

  for line in y : 
    index[int(line[2][0])] = (line[0], line[1])
  return index

t1 = time.time()
index1 = creation_index_unique("data.txt")
print("done in", round(time.time() - t1, 2), "s")

done in 10.16 s


In [ ]:
def creation_index_not_unique(fichier):
  index = {}

  # la clé est la valeur du 1er attribut
  # la valeur est un rowid composé de (page, position)
  y = lecture_sequentielle_par_page(fichier)

  for line in y :
    if(int(line[2][1]) in index):
      index[int(line[2][1])] = index[int(line[2][1])]+[(line[0], line[1])]
    else : 
      index[int(line[2][1])] = [(line[0], line[1])]
  return index

t1 = time.time()
index2 = creation_index_not_unique("data.txt")
print("done in", round(time.time() - t1, 2), "s")

done in 47.39 s


In [ ]:
print(index1[3241396])

(4901, 364)


In [ ]:
print(index2[2466832])

[(773, 198), (3673, 982)]


# Accès par index

## Index unique scan
Accès pour rechercher les tuples dont le 1er attribut a une valeur donnée.

On peut supposer pour simplifier que l'attribut est unique

In [ ]:
def selection_par_index(fichier, index, valeur_recherchee):
  num_page, position = index[valeur_recherchee]
  return lecture_tuple(fichier, num_page, position)

s = np.random.randint(nb_valeurs_distinctes)
print("valeur recherchée :", s)

t1 = time.time()
selection_par_index("data.txt", index1, s)
print("done in", round(time.time() - t1, 2), "s")

valeur recherchée : 47535
47535,2029165,480648,378600,130851,21807,20858,----------------------------------------------------------------------------------------------------

done in 0.0 s


## Index range scan
Accès pour rechercher les tuples dont le 1er attribut a une valeur comprise dans une intervalle donné

In [ ]:
def selection_par_index_plage(fichier, index, borne_inf, borne_sup):  
  for i in range(borne_inf, borne_sup+1):
    num_page, position = index[i]
    lecture_tuple(fichier, num_page, position)



In [ ]:
t1 = time.time()
selection_par_index_plage("data.txt", index1, 17000, 17030)
print("done in", round(time.time() - t1, 2), "s")

17000,329154,809041,317574,195139,106163,33699,----------------------------------------------------------------------------------------------------

17001,1800067,561175,386873,92764,126830,55241,----------------------------------------------------------------------------------------------------

17002,1330589,730950,366475,63887,37642,38448,----------------------------------------------------------------------------------------------------

17003,576225,972738,18936,310825,130641,10735,----------------------------------------------------------------------------------------------------

17004,667507,1184684,106008,199168,141336,70698,----------------------------------------------------------------------------------------------------

17005,323462,872184,538811,269158,352,78080,----------------------------------------------------------------------------------------------------

17006,571709,278015,288258,301906,22279,9144,-----------------------------------------------------------------

#Mise à jour de données




## Sélectionner un tuple et modifier un de ses attributs



In [ ]:
def addN(a0, N):
  """
  Dans la ligne correspondant à a0 on ajoute N à l'élément a1.
  """
  page_dir = page_dir_name(DATA)
  num_page, position = index1[a0]

  lines=[]
  with open(page_dir + f"/page{num_page}", "r") as fpr:
    for i, line in enumerate(fpr):
      if(i==position-1):
        t = line.split(',')
        t[1]=str(int(t[1])+N)
        t=','.join(t)
        lines.append(t)
      else : 
        lines.append(line)

  with open(page_dir + f"/page{num_page}", "w") as fpw:
            fpw.write(''.join(lines))


In [ ]:
#addN(844431,1)

##Modifier l'index en conséquence lorsque l'attibut modifié est indexé


In [ ]:
def updateIndex2(fichier, a0, N):
  """
  Mise à jour dans index2 (index non unique) suite aux modification sur a1 faites dans l'index1 avec la fonction addN
  """
  num_page, position = index1[a0]
  print("tuple après modification: ")
  t = selection_par_index(fichier, index1, a0)
  t = t.split(',')
  new_a1 = int(t[1])
  old_a1 = new_a1 - N
  #suppression 
  values = index2.pop(old_a1)
  values.pop(values.index((num_page,position)))
  index2[old_a1] = values
  #ajout
  if(new_a1 in index2):
    index2[new_a1] = index2[new_a1]+[(num_page, position)]
  else : 
    index2[new_a1] = [(num_page, position)]



In [ ]:
a0 = 1405205

print("Avant : ")
t = selection_par_index(DATA, index1, a0)
t = t.split(',')
old_a1 = int(t[1])
print(index2[old_a1])

addN(a0,1)
updateIndex2(DATA, a0, 1)
new_a1 = old_a1 + 1

print("Après : ")
print(index2[old_a1])
print(index2[new_a1])


Avant : 
1405205,2250787,274590,29725,20523,25102,57084,----------------------------------------------------------------------------------------------------

[(4423, 476), (4537, 459), (4890, 738), (4997, 988)]
tuple après modification: 
1405205,2250788,274590,29725,20523,25102,57084,----------------------------------------------------------------------------------------------------

Après : 
[(4423, 476), (4890, 738), (4997, 988)]
[(464, 415), (711, 518), (1033, 773), (1932, 401), (2977, 862), (3272, 848), (4066, 984), (4790, 437), (4537, 459)]


# Persistence

Dans cette partie, on veut rendre les index persistents en les stockant dans des pages. Cela permet d'utiliser les index plus efficacement sans les recréer entièrement.


## Stocker un index (dans plusieurs pages) pour le reconstruire plus rapidement

Proposer une solution pour stocker les entrées triées d'un index unique dans plusieurs pages avec une taille de page fixée (500 entrées par page,  soit 500 clés + 500 rowid).

In [ ]:
def stockIndex(index):

  if(os.path.exists("indexUnique_pages")):
    sh.rmtree("indexUnique_pages")
  os.makedirs("indexUnique_pages")

  nb_entree_par_page = 500

  lines = []
  i=0
  p=0
  for key in sorted(index.keys()):
    lines.append(str(key)+' '+str(index[key][0])+' '+str(index[key][1]))
      
    # créer une page
    if (i+1) % nb_entree_par_page == 0:
      p += 1
      with open("indexUnique_pages" + f"/page{p}", "w") as fp:
        fp.write('\n'.join(lines))
      lines = []
    
    i+=1
  
  # créer une dernière page, si nécessaire
  if len(lines) > 0:
    p +=1
    with open("indexUnique_pages" + f"/page{p}", "w") as fp:
        fp.write('\n'.join(lines))
  
  return p 

nb_pages_index_unique = stockIndex(index1)

## Stockage d'un index non unique
Proposer une solution pour stocker les entrées triées d'un index non unique dans plusieurs pages avec une taille de page fixée. Dans une page, le total du nombre  de clés + le nombre de rowid ne peut pas dépasser 1000.

In [ ]:
def stockIndexNonUnique(index):
  if(os.path.exists("indexNonUnique_pages")):
    sh.rmtree("indexNonUnique_pages")
  os.makedirs("indexNonUnique_pages")

  nb_cle_et_rowid_par_page = 1000

  lines = []
  k=0
  r=0
  p=0
  for key in sorted(index.keys()):
    s=''
    for v in index[key]:
      s += str(v[0])+' '+str(v[1])+' '
    lines.append(str(key)+' '+s)
      
    # créer une page
    if (k+r+1) % nb_cle_et_rowid_par_page == 0:
      p += 1
      with open("indexNonUnique_pages" + f"/page{p}", "w") as fp:
        fp.write('\n'.join(lines))
      lines = []
    
    k+=1
    r+=len(index[key])
  
  # créer une dernière page, si nécessaire
  if len(lines) > 0:
    p +=1
    with open("indexNonUnique_pages" + f"/page{p}", "w") as fp:
        fp.write('\n'.join(lines))
  
  return p

nb_pages_index_non_unique = stockIndexNonUnique(index2)

## Adapter en conséquence les opérations de modification de l'index

On cherche à adapter ce que l'on a fait précédemment.
On souhaite modifier les index suite à l'ajout de N à a1 (à partir de a0).




In [ ]:
def findPageIndexUnique(a0_rech):
  for p in range(nb_pages_index_unique):
    a=lecture_tuple("indexUnique", p+1,1)
    a=a.split(',')
    a0 = a[0]
    if(a0>a0_rech):
      return p
  return nb_pages_index_unique #la dernière page


In [ ]:
def findPageIndexNonUnique(a1_rech):
  for p in range(nb_pages_index_non_unique):
    a=lecture_tuple("indexNonUnique", p+1,1)
    a=a.split(' ')
    a1 = a[0]
    if(int(a1)>a1_rech):
      return p
  return nb_pages_index_non_unique #la dernière page


In [ ]:
def findRowidIndexUnique(a0_rech):
  p = findPageIndexUnique(a0_rech)
  with open("indexUnique_pages" + f"/page{(p)}", "r") as fp:
    for i, line in enumerate(fp):
      t = line.split(' ')
      if(int(t[0])==a0_rech):
        return t[1],t[2]


In [ ]:
def findRowidIndexNonUnique(a0_rech):
  p = findPageIndexNonUnique(a0_rech)
  with open("indexNonUnique_pages" + f"/page{(p)}", "r") as fp:
    for i, line in enumerate(fp):
      t = line.split(' ')
      if(int(t[0])==a0_rech):
        return t[1:]

In [ ]:
def addN_bis(a0,N):
  p = findPageIndexUnique(a0)
  